In [28]:
# !pip install deeprobust
# !conda install pytorch torchvision torchaudio -c pytorch
import torch
# print(torch.__version__)
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-geometric
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis

import random
from torch.nn import Linear,Sigmoid
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, GraphConv
from torch_geometric.nn import global_mean_pool
from random import sample

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/819003294.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [29]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

In [30]:
import numpy
import torch
%load_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))



from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random

from random import sample
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv

from scipy.sparse import random
from scipy.sparse.linalg import norm

from scipy.sparse import random
from scipy.stats import rv_continuous

from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj
import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/1785052457.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [31]:
from random import sample

In [32]:
# from deeprobust.graph.data import Dataset

In [33]:
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv

In [34]:
import os
os.getcwd()
dataset = os.path.join(os.getcwd(),'Cora')
dataset

'/Users/prakashpal/Desktop/Sandip Sir/Cora'

In [35]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj




dataset= Planetoid(root=dataset, name='Cora')
print(dataset[0])
adj = to_dense_adj(dataset[0].edge_index)
adj = adj[0]
labels = dataset[0].y
labels = labels.numpy()

X = dataset[0].x
X = X.to_dense()
N = X.shape[0]
NO_OF_CLASSES =  len(set(np.array(dataset[0].y)))

print(X.shape, adj.shape)

nn = int(1*N)
X = X[:nn,:]
adj = adj[:nn,:nn]
labels = labels[:nn]
print(X.shape,adj.shape)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
torch.Size([2708, 1433]) torch.Size([2708, 2708])
torch.Size([2708, 1433]) torch.Size([2708, 2708])


In [36]:
A=adj

In [37]:
Data = dataset[0]
Data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [38]:
import torch_geometric.transforms as T

split = T.RandomLinkSplit(
    num_val=0.01,
    num_test=0.02,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0,
)
tra, va, tes = split(Data)

In [39]:
tra

Data(x=[2708, 1433], edge_index=[2, 10242], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[5121], edge_label_index=[2, 5121])

In [40]:
tes

Data(x=[2708, 1433], edge_index=[2, 10346], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[210], edge_label_index=[2, 210])

In [41]:
va

Data(x=[2708, 1433], edge_index=[2, 10242], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[104], edge_label_index=[2, 104])

In [42]:
def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj

theta = get_laplacian(adj)
print(theta.shape)


torch.Size([2708, 2708])


In [43]:

features = X.numpy()
NO_OF_NODES = X.shape[0]
print(features.shape)
print(NO_OF_CLASSES,NO_OF_NODES)

(2708, 1433)
7 2708


In [44]:
def convertScipyToTensor(coo):
  try:
    coo = coo.tocoo()
  except:
    coo = coo
  values = coo.data
  indices = np.vstack((coo.row, coo.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = coo.shape

  return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [45]:
from scipy.sparse import random
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix

p = X.shape[0]
k = int(p*0.5)
n = X.shape[1]
lambda_param = 100
beta_param = 0.1
gamma_param = 100
lr = 1e-5
thresh = 1e-10

from scipy.sparse import random
from scipy.stats import rv_continuous
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
temp = CustomDistribution(seed=1)
temp2 = temp()  # get a frozen version of the distribution
X_tilde = random(k, n, density=0.25, random_state=1, data_rvs=temp2.rvs)
C = random(p, k, density=0.25, random_state=1, data_rvs=temp2.rvs)



In [46]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):

        #print("Checking 1: x", x.shape, "Edge index:", edge_index.shape)
        x = self.conv1(x, edge_index)
        #print("Checking 2: convolution done, new x:", x.shape)
        x = F.relu(x)
        #print("Checking 3: x", x.shape, "training:", self.training)
        x = F.dropout(x, training=self.training)
        #print("Checking 4: dropout done new x", x.shape, "Edge index:", edge_index.shape)
        x = self.conv2(x, edge_index)
        #print("Checking 5: x", x.shape)

        return F.log_softmax(x, dim=1)



In [47]:
features.shape

(2708, 1433)

In [48]:
def feature(feat):
  graph = feat
  print("feat:",graph)
  print("------------------",type(feat))
  # type(feat)

In [49]:
X.shape

torch.Size([2708, 1433])

In [56]:
k_=NO_OF_CLASSES
def experiment_structure(alpha_param,lambda_param,beta_param,gamma_param,C,theta,X,A):
      p = X.shape[0]
      k = int(p*0.5)
      n = X.shape[1]
      ones = csr_matrix(np.ones((k,k)))
      ones = convertScipyToTensor(ones)
      ones = ones.to_dense()
      
      try:
        C = convertScipyToTensor(C)
        C = C.to_dense()
      except:
        C=C
      try:
        theta = convertScipyToTensor(theta)
      except:
        theta = theta
      try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
      except:
        X = X
      # if(torch.cuda.is_available()):
      #   print("GPU is available")
      #   C = C.cuda()
      #   theta = theta.cuda()
      #   X = X.cuda()
      #   ones = ones.cuda()
      
        
      def bracket_term2fun(C,CT,theta):
          # U  = update_U(C,theta).double()
          U = torch.stack(update_U(C, theta)).double()
          UT= torch.transpose(U,0,1)
          Lw = (CT @theta @C).double()
          lb= 1e-5
          ub = 1e+4
          beta = 0.5 
          lambda_ =  laplacian_lambda_update_with_zeroing(lb, ub, beta, U, Lw, k_,C)   
          lambda_matrix =  torch.diag(lambda_,0)
#           print("U size",U.size())
          return U@lambda_matrix@UT
        
      def update_U(C,theta):
            
        CT= torch.transpose(C,0,1)
        product = CT @ theta @ C
        matrix = torch.tensor(product)  
        eigenvalues, eigenvectors = torch.linalg.eig(product)

        # select non-zero eigenvalues and eigenvectors
        non_zero_eigenvalues = []
        non_zero_eigenvectors = []
        for i in range(matrix.shape[0]):
            if matrix[i, i] != 0:
                non_zero_eigenvalues.append(eigenvalues[i])
                non_zero_eigenvectors.append(eigenvectors[:, i])
        U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
        return U    
 

      def update_C(C):
          CT = torch.transpose(C,0,1)
          C.size()
          t1 = alpha_param*(C@ones)
          bracket_term1 = (CT@theta@C)
          bracket_term2 = bracket_term2fun(C,CT,theta) 
          bracket_term = bracket_term1 - bracket_term2   # bracket term (CT*theta*C - U*lambda*UT)
          t22 = -2*(theta@C) 
#           print(t22.type())
          t3 = bracket_term1
          t7 = bracket_term2
          t6 = (CT@A@C)
          t5 = 2* beta_param*(A@C)
          t5 = t5.float()
          t4 = (1.0/k)
          t44 = t4*((torch.ones(k,k)).double())
#           print(t3.device)
#           print(t44.device)
          t8 = (t3 + t44)
          t9 = torch.pinverse(t8)                  #change it
          t9 = t9.float()
#           print(t9.type())
#           print(t9)
          t10 = (t22@t9)
          t11 = (t6 - t7)
          t11 = t11.float()
          t12 = (t5@t11)
          t13 = (t1 + t10 +t12)
        
          #t2 = beta_param*(theta@C@bracket_term.float())
          grad_fc= t13
          C_new=C-gamma_param*grad_fc
          C_new[C_new<thresh] = thresh
          for i in range(len(C_new)):
              C_new[i] = C_new[i]/torch.linalg.norm(C_new[i],1)
          return C_new        
            

        
        
      def laplacian_lambda_update_with_zeroing(lb, ub, beta, U, Lw, k,C):
         # Compute the eigenvalues using the original implementation
        lambda_ = laplacian_lambda_update(lb, ub, beta, U, Lw, k,C)
        # Zero out the first 'k' eigenvalues
        lambda_[:k] = 0
        return lambda_  


      #We set c1 = 10−5 and c2 = 10^4 We observed that the experimental performances of the algorithms 
       #are not sensitive to different values of c1 and c2 as long as they are reasonably small and large,respectively
      # K is the number of smallest eigenvalues of the Laplacian matrix that are being ignored while updating the eigenvalues.
      def laplacian_lambda_update(lb, ub, beta, U, Lw, k, C):
        q = Lw.size(1) - k
        U = U
        UT= torch.transpose(U,0,1)
        UT = UT.type(torch.float64)
        UT = UT
        
        CT= torch.transpose(C,0,1)
        CT = CT.type(torch.float64)
        CT = CT
        
        AC=(A@C).double()
        AC = AC
        
        Af=(CT@AC).double()
        Af = Af
        Af.device
        U.device
        dd = U@Af@UT
        
        product = dd
        matrix = torch.tensor(product)            

        non_zero_diag_elements = []
        for i in range(matrix.shape[0]):
            if matrix[i, i] != 0:
                non_zero_diag_elements.append(matrix[i, i])
            if len(non_zero_diag_elements) == len(matrix):
                break

        k = len(non_zero_diag_elements)
        d = torch.diag(torch.tensor(non_zero_diag_elements))

       
        lambda_ = 0.5 * (d + torch.sqrt(d.pow(2) + 4 / beta))
#         print(lambda_)
        lambda_,indices = torch.sort(lambda_, dim=- 1, descending=True)
        eps = 1
        condition = torch.stack([(lambda_[q] - ub) <= eps,
                         (lambda_[0] - lb) >= -eps]).all(dim=0)

#                                   (lambda_[1:(q)] - lambda_[0:(q-1)]) >= -eps])
        if condition.all():
            return lambda_
        else:
            greater_ub = lambda_ > ub
            lesser_lb = lambda_ < lb
            lambda_[greater_ub] = ub
            lambda_[lesser_lb] = lb
            condition = torch.stack([(lambda_[q] - ub) <= eps,
                         (lambda_[0] - lb) >= -eps]).all(dim=0)

#                                   (lambda_[1:q] - lambda_[0:(q-1)]) >= -eps])
            if condition.all():
                return lambda_
            else:
#                 print(lambda_)
                raise ValueError("eigenvalues are not in increasing order, consider increasing the value of beta")
            

      for i in tqdm(range(20)): #update C only 21
         C = update_C(C)
            

      feat = torch.linalg.pinv(C)@X
      # feature(feat)
      print("shape of feat", feat.shape)

      return feat,C



In [57]:
import torch
import torchvision
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling


class Net1(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=300
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 100 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [58]:
def edge_index_coarsen(C_n,theta,labels,graph):
    from torch_geometric.data import Data
    C_n =C_n.detach().cpu()
    theta = theta.detach().cpu()
    L_c= C_n.T@theta@C_n
    from scipy.sparse import csr_matrix
    Wc=(-1*L_c)*(1-np.eye(L_c.shape[0]))
    Wc[Wc<0.01]=0
    Wc= csr_matrix(Wc)
    Wc = Wc.tocoo()
    row = torch.from_numpy(Wc.row).to(torch.long)
    col = torch.from_numpy(Wc.col).to(torch.long)
    edge_index_coarsen2 = torch.stack([row, col], dim=0)
#     print("Edge_index_coarsen Shape", edge_index_coarsen2.shape)
    return edge_index_coarsen2.shape[1]

In [59]:
def link_prediction_auc(C_n,theta,labels,graph):
    from torch_geometric.data import Data
    C_n =C_n.detach().cpu()
    theta = theta.detach().cpu()
    L_c= C_n.T@theta@C_n
    from scipy.sparse import csr_matrix
    Wc=(-1*L_c)*(1-np.eye(L_c.shape[0]))
    Wc[Wc<0.001]=0
    Wc= csr_matrix(Wc)
    Wc = Wc.tocoo()
    row = torch.from_numpy(Wc.row).to(torch.long)
    col = torch.from_numpy(Wc.col).to(torch.long)
    edge_index_coarsen2 = torch.stack([row, col], dim=0)
    print("Edge_index_coarsen Shape", edge_index_coarsen2.shape)
    
        
    edge_weight = torch.from_numpy(Wc.data)
    def one_hot(x, class_count):
        return torch.eye(class_count)[x, :]

    Y = labels
    Y = one_hot(Y,NO_OF_CLASSES)
    P=np.linalg.pinv(C_n)
    labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)
    data =Data(x=graph , edge_index= edge_index_coarsen2, y=labels_coarse)
    graph = data
    import torch_geometric.transforms as T

    split = T.RandomLinkSplit(
        num_val=0.01,
        num_test=0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0,
    )
    train_data, val_data, test_data = split(graph)
    train_data = train_data.cpu()
    test_data = test_data.cpu()
    val_data = val_data.cpu()

    test_data=tes
    model = Net1(dataset.num_features, 128, 64)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    criterion = torch.nn.BCEWithLogitsLoss()
    model = train_link_predictor(model, train_data, val_data, optimizer, criterion)

    test_auc = eval_link_predictor(model, test_data)
    print(f"Test: {test_auc:.3f}")
    acc = test_auc*100
    acc

In [ ]:

highest_accuracy=0
for alpha_param in [100,10,1,0.1,0.01,0.001]:
    for lambda_param in [100,10,1,0.1,0.01,0.001]:
      for beta_param in [0.001,0.01,0.1,1,10,100]:
          for gamma_param in [100,10,1,0.1,0.01,0.001]:
            feat_reduced,C_tilde = experiment_structure(alpha_param,lambda_param,beta_param,gamma_param,C,theta,X,A)
            edge_index_c= edge_index_coarsen(C_tilde,theta,labels,feat_reduced)
            print(edge_index_c)
            if edge_index_c >300:
                link_prediction_auc(C_tilde,theta,labels,feat_reduced)
            else:    
                print("breaking")


  0%|                                                    | 0/20 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than to

shape of feat torch.Size([1354, 1433])
23554
Edge_index_coarsen Shape torch.Size([2, 105078])
Epoch: 100, Train Loss: 0.511, Val AUC: 0.953
Epoch: 200, Train Loss: 0.421, Val AUC: 0.972
Epoch: 300, Train Loss: 0.406, Val AUC: 0.973
Test: 0.516


  0%|                                                    | 0/20 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than to

shape of feat torch.Size([1354, 1433])
48058
Edge_index_coarsen Shape torch.Size([2, 106564])
Epoch: 100, Train Loss: 27.984, Val AUC: 0.775
Epoch: 200, Train Loss: 1.600, Val AUC: 0.859
Epoch: 300, Train Loss: 0.852, Val AUC: 0.932
Test: 0.673


  0%|                                                    | 0/20 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than to

...

  0%|                                                                           | 0/20 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

shape of feat torch.Size([1354, 1433])
26872
Edge_index_coarsen Shape torch.Size([2, 106284])
Epoch: 100, Train Loss: 0.434, Val AUC: 0.971
Epoch: 200, Train Loss: 0.423, Val AUC: 0.974
Epoch: 300, Train Loss: 0.436, Val AUC: 0.968
Test: 0.538


  0%|                                                                           | 0/20 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

shape of feat torch.Size([1354, 1433])
83990
Edge_index_coarsen Shape torch.Size([2, 235364])
Epoch: 100, Train Loss: 0.491, Val AUC: 0.924
Epoch: 200, Train Loss: 0.468, Val AUC: 0.938
Epoch: 300, Train Loss: 0.460, Val AUC: 0.942
Test: 0.478


  0%|                                                                           | 0/20 [00:00<?, ?it/s]/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matrix = torch.tensor(product)
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  U = [torch.tensor(eigenvector) for eigenvector in non_zero_eigenvectors]
/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_78050/2761122159.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad